# 03 - Data Preprocessing

**Objective**: Clean data and prepare for modeling

**Steps**:
1. Missing value imputation
2. Outlier capping (99th percentile)
3. Feature encoding (Label, One-Hot, Target)
4. Save preprocessing pipeline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from category_encoders import TargetEncoder

print('Libraries loaded!')

In [ ]:
# Paths
RAW_PATH = Path('../data/01_raw')
INTERMEDIATE_PATH = Path('../data/02_intermediate')
PRIMARY_PATH = Path('../data/03_primary')
MODEL_PATH = Path('../data/06_models')

for p in [INTERMEDIATE_PATH, PRIMARY_PATH, MODEL_PATH]:
    p.mkdir(parents=True, exist_ok=True)

# Load data
df_train = pd.read_csv(RAW_PATH / 'cell2celltrain.csv')
df_holdout = pd.read_csv(RAW_PATH / 'cell2cellholdout.csv')

print(f"Train: {df_train.shape}")
print(f"Holdout: {df_holdout.shape}")

In [ ]:
# Define target and features to exclude
TARGET = 'Churn'
EXCLUDE_FEATURES = ['CustomerID', 'RetentionCalls', 'RetentionOffersAccepted', 'MadeCallToRetentionTeam']

print(f"Target: {TARGET}")
print(f"Excluded (leakage): {EXCLUDE_FEATURES}")

## 1. Missing Value Imputation

In [ ]:
def analyze_missing(df):
    """Analyze missing values."""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({'Count': missing, 'Pct': missing_pct})
    return missing_df[missing_df['Count'] > 0].sort_values('Pct', ascending=False)

print("📊 Missing Values (Before):")
display(analyze_missing(df_train))

In [ ]:
def impute_missing(df, create_flags=True, flag_threshold=0.05):
    """Impute missing values with strategy based on % missing."""
    df = df.copy()
    imputation_log = []
    
    for col in df.columns:
        missing_pct = df[col].isnull().sum() / len(df)
        
        if missing_pct == 0:
            continue
            
        # Create flag if > 5% missing
        if create_flags and missing_pct > flag_threshold:
            df[f'{col}_IsMissing'] = df[col].isnull().astype(int)
            imputation_log.append({'Column': col, 'Action': f'Created {col}_IsMissing flag'})
        
        # Impute based on dtype
        if df[col].dtype in ['int64', 'float64']:
            # Numerical: median imputation
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            imputation_log.append({'Column': col, 'Action': f'Median imputed: {median_val:.2f}'})
        else:
            # Categorical: 'Unknown' category
            df[col] = df[col].fillna('Unknown')
            imputation_log.append({'Column': col, 'Action': 'Filled with Unknown'})
    
    return df, pd.DataFrame(imputation_log)

# Apply imputation
df_train_imputed, imputation_log = impute_missing(df_train)
df_holdout_imputed, _ = impute_missing(df_holdout, create_flags=True)

print("📊 Imputation Log:")
display(imputation_log)
print(f"\n✅ Missing values after: {df_train_imputed.isnull().sum().sum()}")

## 2. Outlier Capping (99th Percentile)

In [ ]:
# Features to cap
CAP_FEATURES = ['MonthlyRevenue', 'MonthlyMinutes', 'OverageMinutes', 'RoamingCalls', 'TotalRecurringCharge']
CAP_FEATURES = [f for f in CAP_FEATURES if f in df_train_imputed.columns]

print(f"Features to cap at 99th percentile: {CAP_FEATURES}")

In [ ]:
def cap_outliers(df, features, percentile=0.99):
    """Cap outliers at specified percentile."""
    df = df.copy()
    cap_log = []
    caps = {}
    
    for col in features:
        if col not in df.columns:
            continue
            
        cap_val = df[col].quantile(percentile)
        n_capped = (df[col] > cap_val).sum()
        
        df[col] = df[col].clip(upper=cap_val)
        
        caps[col] = cap_val
        cap_log.append({
            'Feature': col,
            'Cap Value': cap_val,
            'Records Capped': n_capped,
            'Pct Capped': (n_capped / len(df)) * 100
        })
    
    return df, pd.DataFrame(cap_log), caps

# Apply capping
df_train_capped, cap_log, cap_values = cap_outliers(df_train_imputed, CAP_FEATURES)

# Apply same caps to holdout
df_holdout_capped = df_holdout_imputed.copy()
for col, cap in cap_values.items():
    if col in df_holdout_capped.columns:
        df_holdout_capped[col] = df_holdout_capped[col].clip(upper=cap)

print("📊 Outlier Capping Log:")
display(cap_log)

# Save cap values
joblib.dump(cap_values, MODEL_PATH / 'outlier_caps.pkl')
print(f"\n💾 Saved: outlier_caps.pkl")

## 3. Feature Encoding

In [ ]:
# Identify feature types
categorical_cols = df_train_capped.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [c for c in categorical_cols if c not in EXCLUDE_FEATURES]

# Categorize by cardinality
low_cardinality = []  # < 10 unique - One-Hot
high_cardinality = []  # >= 10 unique - Target Encoding

for col in categorical_cols:
    n_unique = df_train_capped[col].nunique()
    if n_unique < 10:
        low_cardinality.append(col)
    else:
        high_cardinality.append(col)

print(f"Low cardinality (One-Hot): {low_cardinality}")
print(f"High cardinality (Target Encoding): {high_cardinality}")

In [ ]:
# One-Hot Encoding for low cardinality
df_train_encoded = df_train_capped.copy()
df_holdout_encoded = df_holdout_capped.copy()

if low_cardinality:
    df_train_encoded = pd.get_dummies(df_train_encoded, columns=low_cardinality, drop_first=True)
    df_holdout_encoded = pd.get_dummies(df_holdout_encoded, columns=low_cardinality, drop_first=True)
    
    # Align columns
    missing_cols = set(df_train_encoded.columns) - set(df_holdout_encoded.columns)
    for col in missing_cols:
        df_holdout_encoded[col] = 0
    df_holdout_encoded = df_holdout_encoded[df_train_encoded.columns.intersection(df_holdout_encoded.columns)]
    
    print(f"✅ One-Hot encoded: {low_cardinality}")
    print(f"   New columns: {len(df_train_encoded.columns) - len(df_train_capped.columns)}")

In [ ]:
# Target Encoding for high cardinality (with fold-based to prevent leakage)
if high_cardinality and TARGET in df_train_encoded.columns:
    target_encoder = TargetEncoder(cols=high_cardinality, smoothing=1.0)
    
    # Fit on training data
    df_train_encoded[high_cardinality] = target_encoder.fit_transform(
        df_train_encoded[high_cardinality], 
        df_train_encoded[TARGET]
    )
    
    # Transform holdout
    for col in high_cardinality:
        if col in df_holdout_encoded.columns:
            df_holdout_encoded[col] = target_encoder.transform(df_holdout_encoded[[col]])
    
    # Save encoder
    joblib.dump(target_encoder, MODEL_PATH / 'target_encoder.pkl')
    print(f"✅ Target encoded: {high_cardinality}")
    print(f"💾 Saved: target_encoder.pkl")

## 4. Remove Excluded Features

In [ ]:
# Remove excluded features
cols_to_drop = [c for c in EXCLUDE_FEATURES if c in df_train_encoded.columns]

df_train_final = df_train_encoded.drop(columns=cols_to_drop, errors='ignore')
df_holdout_final = df_holdout_encoded.drop(columns=cols_to_drop, errors='ignore')

print(f"Dropped columns: {cols_to_drop}")
print(f"\nFinal train shape: {df_train_final.shape}")
print(f"Final holdout shape: {df_holdout_final.shape}")

## 5. Data Validation

In [ ]:
# Validate preprocessing
print("📊 PREPROCESSING VALIDATION")
print("="*60)

# Check for remaining missing values
train_missing = df_train_final.isnull().sum().sum()
print(f"Missing values (train): {train_missing}")

# Check for infinite values
numeric_cols = df_train_final.select_dtypes(include=['int64', 'float64']).columns
inf_count = np.isinf(df_train_final[numeric_cols]).sum().sum()
print(f"Infinite values (train): {inf_count}")

# Check target distribution
if TARGET in df_train_final.columns:
    churn_rate = df_train_final[TARGET].mean() * 100
    print(f"Churn rate: {churn_rate:.2f}%")

# Data types summary
print(f"\nData types:")
print(df_train_final.dtypes.value_counts())

## 6. Save Preprocessed Data

In [ ]:
# Save preprocessed datasets
df_train_final.to_csv(PRIMARY_PATH / 'preprocessed_train.csv', index=False)
df_holdout_final.to_csv(PRIMARY_PATH / 'preprocessed_holdout.csv', index=False)

# Save feature list
feature_cols = [c for c in df_train_final.columns if c != TARGET]
pd.DataFrame({'feature': feature_cols}).to_csv(PRIMARY_PATH / 'feature_list.csv', index=False)

print("💾 Saved:")
print(f"   - {PRIMARY_PATH / 'preprocessed_train.csv'}")
print(f"   - {PRIMARY_PATH / 'preprocessed_holdout.csv'}")
print(f"   - {PRIMARY_PATH / 'feature_list.csv'}")

print(f"\n✅ Features: {len(feature_cols)}")
print(f"✅ Train samples: {len(df_train_final):,}")
print(f"✅ Holdout samples: {len(df_holdout_final):,}")

In [ ]:
print("\n" + "="*60)
print("📋 PREPROCESSING COMPLETE")
print("="*60)
print("\n📌 NEXT: Proceed to 04_Feature_Engineering.ipynb")